In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
# Jenny's codes start here
# vaccine_us_county_clean file is pre-cleaned with selected columns from CDC data

vaccine_county = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)

In [ ]:
vaccine_county_rename = vaccine_county.rename(columns={"Recip_County": "County", "Series_Complete_Pop_Pct": "Vaccination_Complete_Pct","Recip_State":"State"})
vaccine_county_rename.shape

In [ ]:
# Merge US vaccination file with US coord file

us_county = pd.read_csv("Resource/05_us_county_coord.csv",low_memory=False)
us_county_rename = us_county.rename(columns={"fips": "FIPS", "county": "County", "state": "State"})
us_county_coord = us_county_rename[['FIPS','lat','long']]

In [ ]:
vaccine_county_coord = pd.merge(vaccine_county_rename, us_county_coord, on='FIPS')
vaccine_county_coord.shape

In [ ]:
# Merge US vaccination_coord file with SIV file

svi_df = pd.read_csv("Resource/04_siv_county.csv",low_memory=False)
svi_df = svi_df.dropna(subset=['Social Vulnerability Index (SVI)'])
svi_df = svi_df.loc[svi_df['Social Vulnerability Index (SVI)']!=0]
svi_df_rename = svi_df.rename(columns={"FIPS Code": "FIPS", "County Name": "County"})
# svi_df_rename.shape

In [ ]:
svi_df_rename['County'] = svi_df_rename['County'].str.split(',', expand=True)[0]

In [ ]:
vaccine_svi_df = pd.merge(vaccine_county_coord, svi_df_rename, on='FIPS')
vaccine_svi_df = vaccine_svi_df[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','Metro_status','Social Vulnerability Index (SVI)','SVI Category','lat','long']] 
vaccine_svi_df = vaccine_svi_df.rename(columns={"County_x": "County","State_x": "State"})
vaccine_svi_df.shape
# SVI has no Puerto Rico data 

In [ ]:
# Merge US vaccination_coord with US election data to add FIPS code

us_election= pd.read_csv("Resource/03_president_county_candidate.csv",low_memory=False)

In [ ]:
us_election_rename = us_election.rename(columns={"state": "State","county":"County"})
us_county_fips = us_county_rename[['FIPS','State','County']]
us_election_fips = pd.merge(us_election_rename,us_county_rename,how='inner',left_on = ['State','County'], right_on=['State','County'])

# Only keep DEM&REP rows

us_election_fips = us_election_fips.loc[(us_election_fips.party=='DEM')|(us_election_fips.party=='REP')]
us_election_fips.shape

In [ ]:
# Merge US vaccination with US election_fips file

vaccine_election = pd.merge(vaccine_county_rename, us_election_fips, on='FIPS')
vaccine_election_df = vaccine_election[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','candidate','party','total_votes','won','lat','long']]
vaccine_election_df = vaccine_election_df.rename(columns={"County_x": "County","State_x": "State"})
vaccine_election_df.shape

In [ ]:
vaccine_svi_df.to_csv("Clean_data/vaccine_svi_df.csv", encoding="utf-8", index=False)

In [ ]:
vaccine_county_coord.to_csv("Clean_data/vaccine_county_coord.csv", encoding="utf-8", index=False)

In [ ]:
vaccine_election_df.to_csv("Clean_data/vaccine_election_df.csv", encoding="utf-8", index=False)

In [ ]:
# Jenny's codes end here